## Dynamic Plans

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import numpy as np

# Custom modules
from helpers import global2local,local2global, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC,Color
from plan import Plan, State
from oracle import Oracle

kill_processes()

## Create Plans

In [2]:
gazebo_word_path = "simulators/gazebo/worlds/medical_delivery_video.world"  ##
# target_box_center = (-85.263400, 56.518300, 0.593695, 0)
salon_pos=(5.664530, 13.713500, 12, 0)
# burger_pos=(54.816000, 5.409120, 0, 0)
univ_pos=(24.476360, -25.651995, 0, 0)
# house_pos=(0,4.000000,0,0)
# building_pos = (37.711900,-4.700690,0,0)
# origin = (0, 0, 0)

#uav1
uav1_offset = (14.816900,19.156799,0,0)
uav1_1 = (14.816900,19.156799,10)
uav1_2 = (14.816900,-22.570171,10)

#uav2
uav2_offset = (14.816900,-7.962251,0,0)
uav2_1 = (14.816900,-7.962251,12)
uav2_2 = (48.741478,-7.962251,12)

#uav3
uav3_offset = univ_pos
uav3_1 = (24.476360, -25.651995, 9)
uav3_2 = (0,4.000000,9)

#uav4
uav4_offset = (-7.633500, -22.570171, 0, 0)
uav4_1 = (-7.633500, -22.570171, 9)
uav4_2 = (43.856503,-22.570171,9)

#uav5
uav5_offset = salon_pos
uav5_1 = (5.664530, 13.713500, 13)
uav5_2 = (24.476360, -25.651995, 13)


offsets = [  # east, north, up, heading
    uav1_offset,
    uav2_offset ,
    uav3_offset ,
    uav4_offset ,
    uav5_offset ,
]
homes = np.array([offset[:3] for offset in offsets])
n_vehicles = len(offsets)


In [3]:
global_paths=[
    np.array([uav1_1,uav1_2]),
    np.array([uav2_1,uav2_2]),
    np.array([uav3_1,uav3_2]),
    np.array([uav4_1,uav4_2]),
    np.array([uav5_1,uav5_2]),
    ]
local_paths=[global2local(path, home) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,wp_margin=0.5,navegation_speed=6,takeoof_alt=30) for path in local_paths]


## Visualization Parameters

In [4]:
colors = 2*[Color.ORANGE] + 3*[Color.GREEN] 

waypoints = {
    f"city_homes": {"pos": homes[2:], "color": Color.ORANGE},
    f"rural_home": {"pos": homes[:2], "color": Color.GREEN}}

markers = {
    f"waypoint_{i+1}": {"pos": global_paths[i], "color": colors[i]}
    for i in range(n_vehicles)
}

waypoints.update(markers)

plot_3d_interactive(
    markers, title="Simulation Markers", expand=[0.2, 0.2, 0.6], ground=-0.05
)

## Choose Simulator

In [5]:
simulator = Simulator(name="NONE", offsets=offsets, plans=plans)

simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))

simulator = Gazebo(
    offsets=offsets,
    plans=plans,
    world_path=gazebo_word_path,
    models=n_vehicles * ["iris"],
    colors = colors,
    markers={}, #
)

Launch Simulator

In [6]:
uavs = simulator.launch()
for uav in uavs:
    uav.safety_radius = 1
orac = Oracle(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀
Vehicle 3 launched 🚀
Vehicle 4 launched 🚀
Vehicle 5 launched 🚀


## Execute Plan

In [7]:
while orac.vehs:
    orac.gather_broadcasts()
    for uav in orac.vehs.copy():
        orac.update_neighbors(uav)
        if uav.plan.state == State.DONE:
            orac.remove(uav)
        else:
            uav.act()
kill_processes()

Vehicle 1: ▶️ Plan Started: 📋 basic
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 2: ▶️ Plan Started: 📋 basic
Vehicle 2: ▶️ Action Started: 🔧 PREARM
Vehicle 2: ▶️ Step Started: Check disarmed
Vehicle 3: ▶️ Plan Started: 📋 basic
Vehicle 3: ▶️ Action Started: 🔧 PREARM
Vehicle 3: ▶️ Step Started: Check disarmed
Vehicle 4: ▶️ Plan Started: 📋 basic
Vehicle 4: ▶️ Action Started: 🔧 PREARM
Vehicle 4: ▶️ Step Started: Check disarmed
Vehicle 5: ▶️ Plan Started: 📋 basic
Vehicle 5: ▶️ Action Started: 🔧 PREARM
Vehicle 5: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 2: ✅ Step Done: Check disarmed
Vehicle 3: ✅ Step Done: Check disarmed
Vehicle 5: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF
Vehicle 2: ▶️ Step Started: Check EKF
Vehicle 3: ▶️ Step Started: Check EKF
Vehicle 5: ▶️ Step Started: Check EKF
Vehicle 4: ✅ Step Done: Check disarmed
Vehicle 4: ▶️ Step Started: Check EKF
Vehicle 3: ✅ Step Done: C